<a href="https://colab.research.google.com/github/trishnakalita660/python_/blob/main/pySpark_udf%26join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar -xvf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark

tar: spark-3.2.0-bin-hadoop3.2.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 198 kB 45.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=e3be216f3b6f0a32a4242b8a63fd1168072675e5a902b410c369361ebb71a4ca
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [4]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
!wget https://raw.githubusercontent.com/futurexskill/bigdata/master/store_customers.csv

--2022-01-28 05:28:52--  https://raw.githubusercontent.com/futurexskill/bigdata/master/store_customers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 199132 (194K) [text/plain]
Saving to: ‘store_customers.csv’

store_customers.csv 100%[===================>] 194.46K  --.-KB/s    in 0.01s   

2022-01-28 05:28:53 (13.7 MB/s) - ‘store_customers.csv’ saved [199132/199132]



In [6]:
!ls

sample_data  store_customers.csv


In [7]:
data_frame = spark.read.csv('store_customers.csv', header=True)

In [8]:
data_frame.show()

+----------+---+------+------+-------+
|CustomerID|Age|Salary|Gender|Country|
+----------+---+------+------+-------+
|         1| 72| 20000|  Male|Germany|
|         2| 72| 22000|Female| France|
|         3| 70| 24000|Female|England|
|         4| 75|  2600|  Male|England|
|         5| 33| 50000|  Male| France|
|         6| 52| 35000|Female|England|
|         7| 31|  4300|  Male|Germany|
|         8| 37| 32000|Female| France|
|         9| 76| 35000|  Male|Germany|
|        10| 58| 37000|Female| France|
|        11| 70| 25000|  Male|Germany|
|        12| 28| 27000|Female| France|
|        13| 21| 29000|Female|England|
|        14| 34|  7600|  Male|England|
|        15| 45| 55000|  Male| France|
|        16| 32| 40000|Female|England|
|        17| 62|  9300|  Male|Germany|
|        18| 54| 37000|Female| France|
|        19| 33| 40000|  Male|Germany|
|        20| 46| 42000|Female| France|
+----------+---+------+------+-------+
only showing top 20 rows



In [9]:
data_frame.count()

7000

In [10]:
!wget https://raw.githubusercontent.com/futurexskill/bigdata/master/store_transactions.csv

--2022-01-28 05:32:45--  https://raw.githubusercontent.com/futurexskill/bigdata/master/store_transactions.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7257374 (6.9M) [text/plain]
Saving to: ‘store_transactions.csv’

store_transactions. 100%[===================>]   6.92M  --.-KB/s    in 0.05s   

2022-01-28 05:32:45 (129 MB/s) - ‘store_transactions.csv’ saved [7257374/7257374]



In [11]:
transaction = spark.read.csv("store_transactions.csv", header = True)

In [12]:
transaction.count()

1048575

In [14]:
# UDF

from pyspark.sql.functions import udf

year = udf(lambda Date: Date.split('-')[2])

In [15]:
transaction = transaction.withColumn("year",year(transaction.Date))
transaction.show(20)

+----------+---------+------+----------+----+
|CustomerID|ProductID|Amount|      Date|year|
+----------+---------+------+----------+----+
|      3427|        3|  7541|22-11-2019|2019|
|      4378|       14|  7271|15-12-2019|2019|
|      3751|       47|  4276|20-11-2019|2019|
|      6899|      146|  8923|22-11-2019|2019|
|      4561|       46|  4891|30-11-2019|2019|
|      2299|      143|  7545|05-12-2019|2019|
|       553|       43|  1147|08-12-2019|2019|
|      3406|      134|   245|12-12-2019|2019|
|      5278|       34|  8765|09-12-2019|2019|
|      2456|       68|  3820|30-11-2019|2019|
|      6963|       58|  1850|27-11-2019|2019|
|      4700|      140|  3948|27-11-2019|2019|
|      3566|       76|   401|17-12-2019|2019|
|      2212|       38|  4750|16-12-2019|2019|
|      4677|       18|  1149|18-11-2019|2019|
|      4295|       35|  3241|19-11-2019|2019|
|      4228|       91|  6405|01-12-2019|2019|
|      2466|       83|  2978|10-12-2019|2019|
|      3136|       67|  6581|28-11

In [17]:
# JOINS

# by default its inner join

country_amt = data_frame.join(transaction, data_frame.CustomerID == transaction.CustomerID)
country_amt.show()

+----------+---+------+------+-------+----------+---------+------+----------+----+
|CustomerID|Age|Salary|Gender|Country|CustomerID|ProductID|Amount|      Date|year|
+----------+---+------+------+-------+----------+---------+------+----------+----+
|      3427| 69| 14300|Female|England|      3427|        3|  7541|22-11-2019|2019|
|      4378| 78| 42000|  Male|Germany|      4378|       14|  7271|15-12-2019|2019|
|      3751| 57| 45000|Female|England|      3751|       47|  4276|20-11-2019|2019|
|      6899| 34| 12600|  Male|Germany|      6899|      146|  8923|22-11-2019|2019|
|      4561| 37| 50000|Female|Germany|      4561|       46|  4891|30-11-2019|2019|
|      2299| 50| 12600|Female| France|      2299|      143|  7545|05-12-2019|2019|
|       553| 29| 42000|  Male|Germany|       553|       43|  1147|08-12-2019|2019|
|      3406| 37| 35000|Female|England|      3406|      134|   245|12-12-2019|2019|
|      5278| 64| 42000|Female|England|      5278|       34|  8765|09-12-2019|2019|
|   

In [20]:
country_amt.groupBy("Gender").agg({"Amount":"Sum"}).show()

+------+------------+
|Gender| sum(Amount)|
+------+------------+
|Female|2.47260421E8|
|  Male|2.63840709E8|
+------+------------+



In [21]:
!wget https://raw.githubusercontent.com/futurexskill/bigdata/master/store_transactions_mini.csv

--2022-01-28 05:44:46--  https://raw.githubusercontent.com/futurexskill/bigdata/master/store_transactions_mini.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167 [text/plain]
Saving to: ‘store_transactions_mini.csv’

store_transactions_ 100%[===================>]     167  --.-KB/s    in 0s      

2022-01-28 05:44:46 (6.73 MB/s) - ‘store_transactions_mini.csv’ saved [167/167]



In [22]:
!wget https://raw.githubusercontent.com/futurexskill/bigdata/master/store_customers_mini.csv

--2022-01-28 05:45:01--  https://raw.githubusercontent.com/futurexskill/bigdata/master/store_customers_mini.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [text/plain]
Saving to: ‘store_customers_mini.csv’

store_customers_min 100%[===================>]     190  --.-KB/s    in 0s      

2022-01-28 05:45:01 (11.9 MB/s) - ‘store_customers_mini.csv’ saved [190/190]



In [23]:
!ls

sample_data	     store_customers_mini.csv  store_transactions_mini.csv
store_customers.csv  store_transactions.csv


In [24]:
custDfMini = spark.read.csv("store_customers_mini.csv", header = True)
transDfMini = spark.read.csv("store_transactions_mini.csv", header = True)


In [26]:
custDfMini.join(transDfMini, custDfMini.CustomerID == transDfMini.CustomerID).show()

+----------+---+------+------+-------+----------+---------+------+----------+
|CustomerID|Age|Salary|Gender|Country|CustomerID|ProductID|Amount|      Date|
+----------+---+------+------+-------+----------+---------+------+----------+
|         1| 72| 20000|  Male|Germany|         1|        3|  7541|22-11-2019|
|         2| 72| 22000|Female| France|         2|       14|  7271|15-12-2019|
|         5| 33| 50000|  Male| France|         5|       46|  4891|30-11-2019|
|         6| 52| 35000|Female|England|         6|      143|  7545|05-12-2019|
+----------+---+------+------+-------+----------+---------+------+----------+



In [27]:
custDfMini.join(transDfMini, custDfMini.CustomerID == transDfMini.CustomerID, how = "inner").show()

+----------+---+------+------+-------+----------+---------+------+----------+
|CustomerID|Age|Salary|Gender|Country|CustomerID|ProductID|Amount|      Date|
+----------+---+------+------+-------+----------+---------+------+----------+
|         1| 72| 20000|  Male|Germany|         1|        3|  7541|22-11-2019|
|         2| 72| 22000|Female| France|         2|       14|  7271|15-12-2019|
|         5| 33| 50000|  Male| France|         5|       46|  4891|30-11-2019|
|         6| 52| 35000|Female|England|         6|      143|  7545|05-12-2019|
+----------+---+------+------+-------+----------+---------+------+----------+



In [28]:
custDfMini.join(transDfMini, custDfMini.CustomerID == transDfMini.CustomerID, how = "right").show()

+----------+----+------+------+-------+----------+---------+------+----------+
|CustomerID| Age|Salary|Gender|Country|CustomerID|ProductID|Amount|      Date|
+----------+----+------+------+-------+----------+---------+------+----------+
|         1|  72| 20000|  Male|Germany|         1|        3|  7541|22-11-2019|
|         2|  72| 22000|Female| France|         2|       14|  7271|15-12-2019|
|      null|null|  null|  null|   null|         3|       47|  4276|20-11-2019|
|      null|null|  null|  null|   null|         4|      146|  8923|22-11-2019|
|         5|  33| 50000|  Male| France|         5|       46|  4891|30-11-2019|
|         6|  52| 35000|Female|England|         6|      143|  7545|05-12-2019|
+----------+----+------+------+-------+----------+---------+------+----------+



In [29]:
custDfMini.join(transDfMini, custDfMini.CustomerID == transDfMini.CustomerID, how="left").show()

+----------+---+------+------+-------+----------+---------+------+----------+
|CustomerID|Age|Salary|Gender|Country|CustomerID|ProductID|Amount|      Date|
+----------+---+------+------+-------+----------+---------+------+----------+
|         1| 72| 20000|  Male|Germany|         1|        3|  7541|22-11-2019|
|         2| 72| 22000|Female| France|         2|       14|  7271|15-12-2019|
|         5| 33| 50000|  Male| France|         5|       46|  4891|30-11-2019|
|         6| 52| 35000|Female|England|         6|      143|  7545|05-12-2019|
|         7| 31|  4300|  Male|Germany|      null|     null|  null|      null|
|         8| 37| 32000|Female| France|      null|     null|  null|      null|
+----------+---+------+------+-------+----------+---------+------+----------+



In [30]:
custDfMini.join(transDfMini, custDfMini.CustomerID == transDfMini.CustomerID, how = "full").show()

+----------+----+------+------+-------+----------+---------+------+----------+
|CustomerID| Age|Salary|Gender|Country|CustomerID|ProductID|Amount|      Date|
+----------+----+------+------+-------+----------+---------+------+----------+
|         1|  72| 20000|  Male|Germany|         1|        3|  7541|22-11-2019|
|         2|  72| 22000|Female| France|         2|       14|  7271|15-12-2019|
|      null|null|  null|  null|   null|         3|       47|  4276|20-11-2019|
|      null|null|  null|  null|   null|         4|      146|  8923|22-11-2019|
|         5|  33| 50000|  Male| France|         5|       46|  4891|30-11-2019|
|         6|  52| 35000|Female|England|         6|      143|  7545|05-12-2019|
|         7|  31|  4300|  Male|Germany|      null|     null|  null|      null|
|         8|  37| 32000|Female| France|      null|     null|  null|      null|
+----------+----+------+------+-------+----------+---------+------+----------+



In [31]:
custDfMini.join(transDfMini, custDfMini.CustomerID == transDfMini.CustomerID, how = "left_semi").show()

+----------+---+------+------+-------+
|CustomerID|Age|Salary|Gender|Country|
+----------+---+------+------+-------+
|         1| 72| 20000|  Male|Germany|
|         2| 72| 22000|Female| France|
|         5| 33| 50000|  Male| France|
|         6| 52| 35000|Female|England|
+----------+---+------+------+-------+



In [32]:
custDfMini.join(transDfMini, custDfMini.CustomerID == transDfMini.CustomerID, how = "left_anti").show()

+----------+---+------+------+-------+
|CustomerID|Age|Salary|Gender|Country|
+----------+---+------+------+-------+
|         7| 31|  4300|  Male|Germany|
|         8| 37| 32000|Female| France|
+----------+---+------+------+-------+

